In [ ]:
emotions = { '01':'neutral','02':'calm','03':'happy','04':'sad','05':'angry','06':'fear',
            '07':'disgust','08':'surprise'}

In [ ]:
import os

In [ ]:
# extract feature 
#the features that we want to extract are : 
#mfcc
#mel 
#chroma